# Regression Week 5: Feature Selection and LASSO (Interpretation)

In this notebook, you will use LASSO to select features, building on a pre-implemented solver for LASSO (using GraphLab Create, though you can use other solvers). You will:
* Run LASSO with different L1 penalties.
* Choose best L1 penalty using a validation set.
* Choose best L1 penalty using a validation set, with additional constraint on the size of subset.

In the second notebook, you will implement your own LASSO solver, using coordinate descent. 

# Fire up graphlab create

In [1]:
import graphlab

A newer version of GraphLab Create (v1.10.1) is available! Your current version is v1.9.

You can use pip to upgrade the graphlab-create package. For more information see https://dato.com/products/create/upgrade.


# Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [2]:
sales = graphlab.SFrame('kc_house_data.gl/')

2016-06-18 23:16:09,449 [INFO] graphlab.cython.cy_server, 176: GraphLab Create v1.9 started. Logging: /tmp/graphlab_server_1466316955.log


This non-commercial license of GraphLab Create is assigned to parthoruleseverywhere@gmail.com and will expire on May 13, 2017. For commercial licensing options, visit https://dato.com/buy/.


# Create new features

As in Week 2, we consider features that are some transformations of inputs.

In [3]:
from math import log, sqrt
sales['sqft_living_sqrt'] = sales['sqft_living'].apply(sqrt)
sales['sqft_lot_sqrt'] = sales['sqft_lot'].apply(sqrt)
sales['bedrooms_square'] = sales['bedrooms']*sales['bedrooms']

# In the dataset, 'floors' was defined with type string, 
# so we'll convert them to float, before creating a new feature.
sales['floors'] = sales['floors'].astype(float) 
sales['floors_square'] = sales['floors']*sales['floors']

* Squaring bedrooms will increase the separation between not many bedrooms (e.g. 1) and lots of bedrooms (e.g. 4) since 1^2 = 1 but 4^2 = 16. Consequently this variable will mostly affect houses with many bedrooms.
* On the other hand, taking square root of sqft_living will decrease the separation between big house and small house. The owner may not be exactly twice as happy for getting a house that is twice as big.

# Learn regression weights with L1 penalty

Let us fit a model with all the features available, plus the features we just created above.

In [5]:
all_features = ['bedrooms', 'bedrooms_square',
            'bathrooms',
            'sqft_living', 'sqft_living_sqrt',
            'sqft_lot', 'sqft_lot_sqrt',
            'floors', 'floors_square',
            'waterfront', 'view', 'condition', 'grade',
            'sqft_above',
            'sqft_basement',
            'yr_built', 'yr_renovated']

Applying L1 penalty requires adding an extra parameter (`l1_penalty`) to the linear regression call in GraphLab Create. (Other tools may have separate implementations of LASSO.)  Note that it's important to set `l2_penalty=0` to ensure we don't introduce an additional L2 penalty.

In [6]:
model_all = graphlab.linear_regression.create(sales, target='price', features=all_features,
                                              validation_set=None, 
                                              l2_penalty=0., l1_penalty=1e10)

Linear regression:

--------------------------------------------------------

Number of examples          : 21613

Number of features          : 17

Number of unpacked features : 17

...

| 5         | 6        | 0.000002  | 1.945254     | 6869667.895833     | 385998.458623 |

| 6         | 7        | 0.000002  | 1.998935     | 6847177.773672     | 380824.455891 |

+-----------+----------+-----------+--------------+--------------------+---------------+

TERMINATED: Iteration limit reached.

This model may not be optimal. To improve it, consider increasing `max_iterations`.

Find what features had non-zero weight.

In [38]:
model_all.coefficients

name,index,value,stderr
(intercept),None,274873.05595,None
bedrooms,None,0.0,None
bedrooms_square,None,0.0,None
bathrooms,None,8468.53108691,None
sqft_living,None,24.4207209824,None
sqft_living_sqrt,None,350.060553386,None
sqft_lot,None,0.0,None
sqft_lot_sqrt,None,0.0,None
floors,None,0.0,None
floors_square,None,0.0,None


In [46]:
import numpy as np
def model_nonzero_coefficients(model):
    w = list(model.coefficients['value'])
    w.reverse()
    return np.poly1d(w)
model_nonzero_coefficients(model_all)

poly1d([  2.00247224e+01,   8.42068035e+02,   0.00000000e+00,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         3.50060553e+02,   2.44207210e+01,   8.46853109e+03,
         0.00000000e+00,   0.00000000e+00,   2.74873056e+05])

In [ ]:
     14         13         5         4        3
20.02 x  + 842.1 x  + 350.1 x + 24.42 x + 8469 x + 2.749e+05

Note that a majority of the weights have been set to zero. So by setting an L1 penalty that's large enough, we are performing a subset selection. 

***QUIZ QUESTION***:
According to this list of weights, which of the features have been chosen? 

# Selecting an L1 penalty

To find a good L1 penalty, we will explore multiple values using a validation set. Let us do three way split into train, validation, and test sets:
* Split our sales data into 2 sets: training and test
* Further split our training data into two sets: train, validation

Be *very* careful that you use seed = 1 to ensure you get the same answer!

In [10]:
(training_and_validation, testing) = sales.random_split(.9,seed=1) # initial train/test split
(training, validation) = training_and_validation.random_split(0.5, seed=1) # split training into train and validate

Next, we write a loop that does the following:
* For `l1_penalty` in [10^1, 10^1.5, 10^2, 10^2.5, ..., 10^7] (to get this in Python, type `np.logspace(1, 7, num=13)`.)
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list.
    * Compute the RSS on VALIDATION data (here you will want to use `.predict()`) for that `l1_penalty`
* Report which `l1_penalty` produced the lowest RSS on validation data.

When you call `linear_regression.create()` make sure you set `validation_set = None`.

Note: you can turn off the print out of `linear_regression.create()` with `verbose = False`

In [23]:
def polynomial_lasso_regression(traindata, l1_penalty):
    model = graphlab.linear_regression.create(traindata, target='price', features=all_features,
                                              validation_set=None, 
                                              l2_penalty=0., l1_penalty=l1_penalty,verbose = False)
    return model

In [34]:
RSS, penalty = float("inf"), 0.0
for l1_penalty in np.logspace(1, 7, num=13):
    model = polynomial_lasso_regression(training, l1_penalty=l1_penalty)
    data_predictions = model.predict(validation)
    RSS_errors = data_predictions - validation['price']
    RSS_test = (RSS_errors  * RSS_errors ).sum()
    if RSS_test < RSS :
        RSS, penalty, nonzero = RSS_test,l1_penalty, (model.coefficients['value']).nnz()
print RSS, penalty, nonzero
    

6.25766285142e+14 10.0 18


*** QUIZ QUESTIONS ***
1. What was the best value for the `l1_penalty`?
2. What is the RSS on TEST data of the model with the best `l1_penalty`?

In [ ]:
6.25766285142e+14

10.0

***QUIZ QUESTION***
Also, using this value of L1 penalty, how many nonzero weights do you have?

In [ ]:
18

# Limit the number of nonzero weights

What if we absolutely wanted to limit ourselves to, say, 7 features? This may be important if we want to derive "a rule of thumb" --- an interpretable model that has only a few features in them.

In this section, you are going to implement a simple, two phase procedure to achive this goal:
1. Explore a large range of `l1_penalty` values to find a narrow region of `l1_penalty` values where models are likely to have the desired number of non-zero weights.
2. Further explore the narrow region you found to find a good value for `l1_penalty` that achieves the desired sparsity.  Here, we will again use a validation set to choose the best value for `l1_penalty`.

In [36]:
max_nonzeros = 7

## Exploring the larger range of values to find a narrow range with the desired sparsity

Let's define a wide range of possible `l1_penalty_values`:

In [37]:
l1_penalty_values = np.logspace(8, 10, num=20)

Now, implement a loop that search through this space of possible `l1_penalty` values:

* For `l1_penalty` in `np.logspace(8, 10, num=20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Extract the weights of the model and count the number of nonzeros. Save the number of nonzeros to a list.
        * *Hint: `model['coefficients']['value']` gives you an SArray with the parameters you learned.  If you call the method `.nnz()` on it, you will find the number of non-zero parameters!* 

In [39]:
non_zero = []
weights = graphlab.SFrame()
for l1_penalty in  np.logspace(8, 10, num=20):
    model = polynomial_lasso_regression(training, l1_penalty=l1_penalty)
    weights.add_column(model.coefficients['value'], name=str(l1_penalty))
    non_zero.append((model.coefficients['value']).nnz())

In [42]:
print weights
print non_zero

+------------------+------------------+------------------+------------------+
|   100000000.0    |   127427498.57   |  162377673.919   |  206913808.111   |
+------------------+------------------+------------------+------------------+
|  25090.9173672   |  26746.6619366   |  28873.1810166   |  31564.6064733   |
|   7789.1770611   |  7743.97904785   |  7691.04707569   |  7618.56776464   |
|  847.559686943   |  822.358945251   |  790.917579684   |  750.170954883   |
|  25234.2091945   |  25178.6259306   |  25115.2785345   |  25026.1774076   |
|  39.0394459636   |  39.0107181353   |  38.9820788132   |  38.9359152531   |
|  1117.31189557   |  1114.91071592   |  1112.23941465   |  1108.38631937   |
| -0.0256861182399 | -0.0186630737228 | -0.0247373605808 | -0.0177447627514 |
|   143.98899197   |  142.519797841   |  140.945844751   |  138.385325946   |
|  20695.3592396   |  20545.8673047   |  20365.2658969   |   20124.74672    |
|  12466.6906503   |  12339.2452502   |  12181.1862577   |  1197

Out of this large range, we want to find the two ends of our desired narrow range of `l1_penalty`.  At one end, we will have `l1_penalty` values that have too few non-zeros, and at the other end, we will have an `l1_penalty` that has too many non-zeros.  

More formally, find:
* The largest `l1_penalty` that has more non-zeros than `max_nonzero` (if we pick a penalty smaller than this value, we will definitely have too many non-zero weights)
    * Store this value in the variable `l1_penalty_min` (we will use it later)
* The smallest `l1_penalty` that has fewer non-zeros than `max_nonzero` (if we pick a penalty larger than this value, we will definitely have too few non-zero weights)
    * Store this value in the variable `l1_penalty_max` (we will use it later)


*Hint: there are many ways to do this, e.g.:*
* Programmatically within the loop above
* Creating a list with the number of non-zeros for each value of `l1_penalty` and inspecting it to find the appropriate boundaries.

In [44]:
l1_penalty_min = 2976351441.63
l1_penalty_max = 3792690190.73  

***QUIZ QUESTIONS***

What values did you find for `l1_penalty_min` and`l1_penalty_max`? 

## Exploring the narrow range of values to find the solution with the right number of non-zeros that has lowest RSS on the validation set 

We will now explore the narrow region of `l1_penalty` values we found:

In [47]:
l1_penalty_values = np.linspace(l1_penalty_min,l1_penalty_max,20)    

* For `l1_penalty` in `np.linspace(l1_penalty_min,l1_penalty_max,20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Measure the RSS of the learned model on the VALIDATION set

Find the model that the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzero`.

In [69]:
penalty = []
nonzero = []
zeroo = []
for l1_penalty in np.linspace(l1_penalty_min,l1_penalty_max,20) :
    model = polynomial_lasso_regression(training, l1_penalty=l1_penalty)
    data_predictions = model.predict(validation)
    RSS_errors = data_predictions - validation['price']
    RSS_test = (RSS_errors  * RSS_errors ).sum()
    penalty.append(l1_penalty) 
    nonzero.append(model_nonzero_coefficients(model))
    zeroo.append( (model.coefficients['value']).nnz())

In [71]:
print penalty[-9]
print nonzero[-9]
print zeroo[-9]

3448968612.16
       14        13         5         4             3
30.01 x  + 2899 x  + 690.1 x + 32.41 x + 1.587e+04 x + 661.7 x + 2.223e+05
7


***QUIZ QUESTIONS***
1. What value of `l1_penalty` in our narrow range has the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzeros`?
2. What features in this model have non-zero coefficients?

In [ ]:
all_features = ['bedrooms', 'bedrooms_square',
            'bathrooms',
            'sqft_living', 'sqft_living_sqrt',
            'sqft_lot', 'sqft_lot_sqrt',
            'floors', 'floors_square',
            'waterfront', 'view', 'condition', 'grade',
            'sqft_above',
            'sqft_basement',
            'yr_built', 'yr_renovated']

3577864204.12
      14        13         5         4             3
29.1 x  + 2670 x  + 655.1 x + 31.76 x + 1.515e+04 x + 149.4 x + 2.291e+05
7